In [6]:
from abc import (ABCMeta, abstractmethod, ABC)
import itertools as it
import functools as ft

def identity(x):
    return x

class Functoid:
    def __init__(self, function):
        self._func = function

    def __call__(self, *args, **kwargs):
        return self._func(*args, **kwargs)

    def curry(self, *args, **kwargs):
        return Functoid(ft.partial(self, *args, **kwargs))
    
    def before(self, f):
        return Functoid(lambda *args, **kwargs: f(self._func(*args, **kwargs)))

    def after(self, f):
        return Functoid(lambda *args, **kwargs: self._func(f(*args, **kwargs)))
    
    def uncurry(self):
        return Functoid(lambda args: self._func(*args))


class FunctoidalClass(type):
    def __new__(cls, name, bases, attrs):
        attrs2 = {k:(Functoid(f) if isfunction(f) else f) for (k,f) in attrs.items()}
        return super(FunctoidalClass, cls).__new__(cls,name,bases,attrs2)
    


class Functor(ABC):
    @Functoid
    @abstractmethod
    def fmap(self, f):
        raise NotImplementedError()


class Applicative(Functor):
    @classmethod
    @abstractmethod
    def pure(cls, x):
        return cls(x)
    
    @abstractmethod
    def ap(self, fa):
        raise NotImplementedError()

    @abstractmethod
    def fmap(self, f):
        return self.pure(f).ap(self)

    
class Monad(Applicative):
    @abstractmethod
    def bind(self, f):
        raise NotImplementedError()

    def fmap(self, f):
        return self.bind(Functoid(f).before(self.pure))
    
    
class Monoid:
    def __init__(self, op, identity):
        self.op = op
        self.identity = identity
    
    def __call__(self, x, y):
        return self.op(x, y)

    def fold(self, iterable):
        return reduce(self.op, iterable, self.identity)

    
class ArrayList(list, Monad):
    def __init__(self, iterable):
        super().__init__(iterable)

    def fmap(self, f):
        return ArrayList(map(f, self))

    @classmethod
    def pure(cls, x):
        return ArrayList((x,))

    def bind(self, f):
        return ArrayList(ft.reduce(lambda y, x: it.chain(y, f(x)), self, []))



In [70]:
import operator as op

class Cons(tuple):
    __slots__ = []
    def __new__(cls, car, cdr):
        return super().__new__(cls, (car, cdr))

    @property
    def car(self):
        return tuple.__getitem__(self, 0)

    @property
    def cdr(self):
        return tuple.__getitem__(self, 1)

    def __repr__(self):
        return "Cons({},{})".format(self.car,self.cdr)
    
    

    
    
class List(Cons, Monad):    
    class Empty:
        __slots__ = []
        class __Empty:
            __slots__ = []
            def __new__(cls):
                return super().__new__(cls)

            def __repr__(self):
                return "Empty"

            def __str__(self):
                return "Empty"
            
            def __iter__(self):
                while False:
                    yield None
                    
            def __len__(self):
                return 0
            
            @property
            def tail(self):
                raise AttributeError("Empty list has no tail")
            
            @property
            def head(self):
                raise AttributeError("Empty list has no head")
                
            def is_empty(self):
                return True
            
            def prepend(self, x):
                return List(x)
            
            def __add__(self, other):
                return other
                
                
        _instance = __Empty()

        def __new__(cls):
            return cls._instance
    
    def __new__(cls, *elements):
        return ft.reduce(lambda y,x: super(List, cls).__new__(cls, x, y), reversed(elements), List.Empty())
        
    @classmethod
    def from_iterable(cls, iterable):
        if isinstance(iterable, cls):
            return iterable
        else:
            return cls(*iterable)

    
    @property
    def head(self):
        return self.car
        
    @property
    def tail(self):
        return self.cdr
    
    def __iter__(self):
        yield self.head
        for x in self.tail:
            yield x
    
    def is_empty(self):
        return False

    def __len__(self):
        return sum(1 for x in self)
    
    def __getitem__(self, index):
        for (i, x) in enumerate(self):
            if i == index:
                return x
        else:
            raise Exception("List index out of bounds")
    
    def __repr__(self):
        return "List({})".format(", ".join(map(str, iter(self))))
        
    
    @classmethod
    def cons(cls, a, b):
        return b.prepend(a)
        
    def prepend(self, x):
        return super(List, self).__new__(type(self), x, self)
    
    def __add__(self, other):
        return ft.reduce(type(self).prepend, reversed(self), other)
    
    def pure(cls, x):
        return List.Empty().prepend(x)
    
    def bind(self, f):
        return ft.reduce(lambda y,x: y+f(x), self, List.Empty())
    
    def fmap(self, f):
        return List.from_iterable(map(f,self))
    
    def ap(self, other):
        return List.from_iterable(f(x) for f in self for x in other)
    

In [71]:
a = List(1,2,3)
print(a.head)
print(a.tail.head)
print(a.tail.tail.head)
a.tail.tail.tail

1
2
3


Empty

In [72]:
b = List.Empty().prepend(1).prepend(2).prepend(3)
print(b)
print(list(reversed(b)))


List(3, 2, 1)
[1, 2, 3]


In [73]:
for x in List(1,2,3):
    print(x)

1
2
3


In [74]:
c = List(1,2,3,4).fmap(lambda x: x*2)
print(c)
d = List(1,2,3,4,5).fmap(Functoid(op.add).curry).ap(List(10,20,30,40,50))
print(d)

List(2, 4, 6, 8)
List(11, 21, 31, 41, 51, 12, 22, 32, 42, 52, 13, 23, 33, 43, 53, 14, 24, 34, 44, 54, 15, 25, 35, 45, 55)


In [49]:
f = Functoid(op.add)

3